### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import array as array
from scipy.stats import pearsonr
%matplotlib inline
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
#pd.set_option("display.max_rows", None, "display.max_columns", None)

### Data Loading

In [4]:
demographics=pd.read_csv("data/election/demographic.csv")
c_results=pd.read_csv("data/output/countyData.csv")
s_results=pd.read_csv("data/output/stateData.csv")
print('County result:',c_results.shape)
print('State result:',s_results.shape)
print('demographic:',demographics.shape)

County result: (4682, 7)
State result: (64, 5)
demographic: (3220, 37)


### Data Cleaning - Demographics

In [3]:
#Porto Rico doesnt vote
demographics.drop(demographics[demographics.State=='Puerto Rico'].index, inplace=True)

#Drop Columns
demographics.drop(['CountyId','IncomeErr','IncomePerCapErr'], axis=1, inplace=True)

#General Data adjustment for mapping
demographics['County'] = demographics['County'].str.replace(' County', '')
demographics['County'] = demographics['County'].str.replace(' Parish', '')
demographics['County'] = demographics['County'].str.replace(' parish', '')

#Virginia
m = demographics['State'] == 'Virginia'
demographics.loc[m, 'County'] = demographics.loc[m, 'County'].replace({' city': ''}, regex=True)
demographics.loc[m, 'County'] = demographics.loc[m, 'County'].replace({' City': ''}, regex=True)

#Alaska
m = demographics['State'] == 'Alaska'
demographics.loc[m, 'County'] = 'Alaska'

#Alaska Merge

#Sorting
demographics.sort_values(by=['State', 'County'], inplace=True, ascending=True)

### Data Cleaning - c_result

In [4]:
#Drop NA for Now until fix
c_results=c_results.dropna()

#TEMPORARY FIX /// Drop all Alaska Except one
c_results.drop(c_results[(c_results.state=='Alaska') & (c_results.county_id > 13)].index, inplace=True)

#Sorting
#c_results.sort_values(by=['state', 'county'], inplace=True, ascending=True)

### Data Cleaning - s_result

In [5]:
#Drop NA for Now until fix
s_results=s_results.dropna()

#Sorting
#c_results.sort_values(by=['state', 'county'], inplace=True, ascending=True)

### Data County Merge

In [6]:
County_merged=pd.merge(right=demographics, left=c_results, right_on=['State','County'], left_on=['state','county'])
County_merged.drop(['State', 'County'], axis=1, inplace=True)
County_merged.shape
County_merged.to_csv (r'data/output/Merged_County_V1.csv', index = False, header=True)

### Data State Merge

In [7]:
def sumMerge(df, adding_param, state_iter):
    return df.loc[df.State==state_iter][adding_param].sum()

def weightedAverageMerge(df, mean_variable):
    totalW=0
    param=0
    for param, weight in zip(df[mean_variable], df['TotalPop']):
        param += float(param) * float(weight)
        totalW += weight
    return param/totalW

In [8]:
#Merge the data that needs to be summed
column_to_sum_list = ['TotalPop','Men', 'Women']
states_to_list = demographics['State'].unique()

for a in column_to_sum_list:
    mylist = []
    for b in states_to_list:
        mylist.append(sumMerge(demographics, a, b))
    s_results[a] = mylist
    #print(len(mylist))

In [9]:
s_results 
s_results.to_csv (r'data/output/Merged_State_V1.csv', index = False, header=True)

### NEED SOME WORK FOR WEIGHTED AVERAGE

#### 1rst try with my own function

In [10]:
#Merge the data that needs to be averaged
weightedAverageMerge(County_merged.loc[County_merged.state == 'Alaska'], 'Income')

#Results Are too Low. I must have made a mistake

279.27782388821566

#### 2nd Try

In [11]:
g = demographics.groupby('State')
demographics['wa']=demographics.Income / g.Income.transform("sum") * demographics.TotalPop
g.wa.sum()

#Results Are too High. I must have made a mistake

State
Alabama                  85052.526500
Alaska                   29045.053217
Arizona                 536780.607774
Arkansas                 45512.761175
California              759714.564592
Colorado                104667.028150
Connecticut             452467.621496
Delaware                328418.178645
District of Columbia    672391.000000
Florida                 328009.341073
Georgia                  81288.843396
Hawaii                  312951.801911
Idaho                    41917.211964
Illinois                149863.480248
Indiana                  73853.129842
Iowa                     33685.296036
Kansas                   31706.252247
Kentucky                 42469.469863
Louisiana                81081.338100
Maine                    90348.393569
Maryland                280782.108812
Massachusetts           517320.187104
Michigan                135826.429494
Minnesota                74080.791981
Mississippi              42143.596717
Missouri                 62701.448805
Montan

#### 3rd Try

In [12]:
stateData = County_merged.copy()
g = stateData.groupby('state')
for state, group in g:
    #     do for all columns: merge
    newVal = incomeMerge(group)
    stateData.loc[(stateData['state'] == state), 'IncomePerCap'] = newVal
    newPop = totalPopMerge(group)
    stateData.loc[(stateData['state'] == state), 'IncomePerCap'] = newVal
stateData = stateData.drop_duplicates()

NameError: name 'incomeMerge' is not defined

In [ ]:
stateData